In [1]:
%load_ext autoreload
%autoreload 2

# Making systems

In [2]:
import molsysmt as msm

In [3]:
from molsysmt import systems
import numpy as np

In [4]:
molsys = msm.convert('1BRS')

[2686 2687 2688 2689]


In [5]:
alt_loc = msm.get('1BRS', alternate_location=True)

TypeError: get_alternate_location_from_system() got an unexpected keyword argument 'structure_indices'

In [ ]:
    alt_loc = msm.get('1BRS', alternate_location=True)
    assert np.all(list(alt_loc[0].keys())==[2686,2687])

    alt_loc_1 = alt_loc[0][2686]
    assert np.all(alt_loc_1['location_id']==np.array(['A','B']))
    assert np.all(alt_loc_1['occupancy']==np.array([0.5,0.5]))
    assert np.all(alt_loc_1['atom_id']==np.array([2687,2688]))
    aux_coors = puw.quantity(np.array([[3.2742, 2.2579, 0.1536], [3.2757, 2.2571, 0.1533]]),'nm')
    assert puw.are_close(alt_loc_1['coordinates'], aux_coors)
    aux_b_factor = puw.quantity(np.array([0.2466, 0.2467]),'nm**2')
    assert puw.are_close(alt_loc_1['b_factor'], aux_b_factor)

    alt_loc_2 = alt_loc[0][2687]
    assert np.all(alt_loc_2['location_id']==np.array(['A','B']))
    assert np.all(alt_loc_2['occupancy']==np.array([0.5,0.5]))
    assert np.all(alt_loc_2['atom_id']==np.array([2689,2690]))
    aux_coors = puw.quantity(np.array([[3.1412, 2.241 , 0.1076], [3.3396, 2.192 , 0.2619]]),'nm')
    assert puw.are_close(alt_loc_2['coordinates'], aux_coors)
    aux_b_factor = puw.quantity(np.array([0.2594, 0.2596]),'nm**2')
    assert puw.are_close(alt_loc_2['b_factor'], aux_b_factor)
